In [20]:
import numpy as np 
import torch
import torch.nn as nn
import pandas as pd
from torch import optim
from torch.utils.data import DataLoader, Dataset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import cv2
import argparse
import csv
import time
import sys
import os
# other library
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
# PyTorch library
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data 
from sklearn.manifold import TSNE
import torch
import torchvision
from torch import nn
from torch import autograd
from torch import optim

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        # define: encoder
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.01, inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.01, inplace=True),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 2),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01, inplace=True),
        )
        # generate mean var
        self.fnc = nn.Sequential(
            nn.Linear(576, 256),
            nn.Linear(256, 128),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        #print(x.shape)
        x1 = self.conv1(x)
        #print(x1.shape)
        x2 = self.conv2(x1)
        #print(x2.shape)
        x3 = self.conv3(x2)
        #print(x3.shape)
        latent = x3.view(len(x3), -1)
        #print(latent.shape)
        predict = self.fnc(latent)
        return predict
        
        
       

In [22]:
class VAE(nn.Module):
    
    def __init__(self):
        super(VAE, self).__init__()

        # define: encoder
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(0.01, inplace=True),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.01, inplace=True),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01, inplace=True),
        )
        
        
        # generate mean var
        self.fc1 = nn.Linear(1600, 512)
        self.fc2 = nn.Linear(1600, 512)
        
        
        
        # define: decoder
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 5, 2, dilation=2),
            nn.LeakyReLU(0.01, inplace=True),
        )

        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(64, 16, 4, 2, dilation=2),
            nn.LeakyReLU(0.01, inplace=True),
        )

        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(16, 3, 6, 1, dilation=1),
            nn.Sigmoid(),
        )


    def encoder(self, x):
        x1 = self.conv1(x)
        #print(x1.shape)
        x2 = self.conv2(x1)
        #print(x2.shape)
        x3 = self.conv3(x2)
        #print(x3.shape)
        return x3
    
    def bottleneck(self,latent):
        latent = latent.view(len(latent), -1)
        mean = self.fc1(latent)
        var = self.fc2(latent)
        z = self.reparameterize(mean,var)
        #print(z.shape)
        return z, mean, var
    
    
    
    def reparameterize(self, mean, var):
        std = var.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mean.size())
        z = mean + std * esp
        return z
    
    

    def decoder(self, z):
        z =  z.view(-1, 128, 2, 2)
        x4 = self.decoder1(z)
        #print(x4.shape)
        x5 = self.decoder2(x4)
        #print(x5.shape)
        x6 = self.decoder3(x5)
        #print(x6.shape)
        #print(x6)
        return x6
    
    def forward(self, x):
        latent = self.encoder(x)
        z , mean, var = self.bottleneck(latent) 
        rec_ = self.decoder(z)
        #print("fake",rec_.shape)
        return rec_, mean, var

In [23]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    device =torch.device('cuda:0')
else:
    device =torch.device("cpu")


# load data and normalize to [-1, 1]
trainX = np.load('./trainX.npy')
print(trainX.shape)
trainX = np.transpose(trainX, (0, 3, 1, 2))/ 255.*2 -1
trainX = torch.Tensor(trainX)


# if use_gpu, send model / data to GPU.
if use_gpu:
    autoencoder.cuda()
    trainX = trainX.cuda()

# Dataloader: train shuffle = True
train_dataloader = DataLoader(trainX, batch_size=32, shuffle=True)
test_dataloader = DataLoader(trainX, batch_size=32, shuffle=False)

(9000, 32, 32, 3)


In [24]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    # BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD



def loss_function(recon_x, x, mu, logvar):
#     BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
    
    loss = nn.L1Loss(reduction='sum')
#     MSE = F.mse_loss(recon_x, x, size_average=False)
    l1_loss = loss(recon_x, x)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return l1_loss + KLD, KLD, l1_loss

In [25]:
def calc_gradient_penalty(netD, real_data, fake_data):
    # print "real_data: ", real_data.size(), fake_data.size()
    alpha = torch.rand(real_data.shape[0], 1)
    alpha = alpha.expand(real_data.shape[0], round(real_data.nelement()/real_data.shape[0])).contiguous().view(real_data.shape[0], 3, 32, 32)
    alpha = alpha.cuda(gpu) if use_cuda else alpha
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)
    
    if use_cuda:
        interpolates = interpolates.cuda(gpu)
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).cuda(gpu) if use_cuda else torch.ones(
                                  disc_interpolates.size()),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * 10
    return gradient_penalty

In [26]:
netG = VAE()
netD = Discriminator()
print(netG)
print(netD)
use_cuda = torch.cuda.is_available()
if use_cuda:
    gpu = 0
if use_cuda:
    Gnet = Gnet.to(device)
    Dnet = Dnet.to(device)


VAE(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace)
  )
  (fc1): Linear(in_features=1600, out_features=512, bias=True)
  (fc2): Linear(in_features=1600, out_features=512, bias=True)
  (decoder1): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(5, 5), stride=(2, 2), dilation=(2, 2))
    (1): LeakyReLU(negative_slope=0.01, inplace)
  )
  (decoder2): Sequential(
    (0)

In [27]:
# We set criterion : L1 loss (or Mean Absolute Error, MAE)
one = torch.FloatTensor([1])
mone = one * -1

if use_cuda:
    one = one.cuda(gpu)
    mone = mone.cuda(gpu)

optimizerG = torch.optim.Adam(netG.parameters(), lr=0.001, weight_decay=1e-5)
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.001, weight_decay=1e-5)

for iteration in range(100):
    print(iteration)
    G_loss = 0
    D_loss = 0
    ############################
    # (1) Update D network
    ###########################
    for idx, image in enumerate(train_dataloader):
            d_loss = 0
            g_loss = 0
            for p in netD.parameters():  # reset requires_grad
                p.requires_grad = True  # they are set to False below in netG update

            _data = image
            netD.zero_grad()


            # import torchvision
            # filename = os.path.join("test_train_data", str(iteration) + str(i) + ".jpg")
            # torchvision.utils.save_image(real_data, filename)

            D_real = netD(image)
            D_real = D_real.mean()
            D_real.backward(mone)

            # train with fake

            fake = autograd.Variable(netG(image)[0].data)
            inputv = fake
            D_fake = netD(inputv)
            D_fake = D_fake.mean()
            D_fake.backward(one)

            # train with gradient penalty
            gradient_penalty = calc_gradient_penalty(netD, image.data, fake.data)
            d_loss=gradient_penalty
            D_loss+=gradient_penalty
            gradient_penalty.backward()

            # print "gradien_penalty: ", gradient_penalty

            D_cost = D_fake - D_real + gradient_penalty
            Wasserstein_D = D_real - D_fake
            optimizerD.step()

            ############################
            # (2) Update G network
            ###########################
            for p in netD.parameters():
                p.requires_grad = False  # to avoid computation
            netG.zero_grad()


            fake = netG(image)[0]
            G = netD(fake)
            G = G.mean()
            G.backward(mone)
            G_cost = -G
            optimizerG.step()
            print(d_loss,end='\r')

0
1ensor(0.2626, grad_fn=<MulBackward0>)
2ensor(0.4061, grad_fn=<MulBackward0>)
3ensor(0.3649, grad_fn=<MulBackward0>)
4ensor(0.2662, grad_fn=<MulBackward0>)
5ensor(0.2437, grad_fn=<MulBackward0>)
6ensor(0.1523, grad_fn=<MulBackward0>)
7ensor(0.2276, grad_fn=<MulBackward0>)
8ensor(0.0419, grad_fn=<MulBackward0>)
9ensor(0.3675, grad_fn=<MulBackward0>)
10nsor(0.1044, grad_fn=<MulBackward0>)
11nsor(0.1611, grad_fn=<MulBackward0>)
12nsor(0.0641, grad_fn=<MulBackward0>)
13nsor(0.1988, grad_fn=<MulBackward0>)
14nsor(0.0642, grad_fn=<MulBackward0>)
15nsor(0.4570, grad_fn=<MulBackward0>)
16nsor(0.2234, grad_fn=<MulBackward0>)
17nsor(0.1021, grad_fn=<MulBackward0>)
18nsor(0.1947, grad_fn=<MulBackward0>)
19nsor(0.1739, grad_fn=<MulBackward0>)
20nsor(0.0921, grad_fn=<MulBackward0>)
21nsor(1.6655, grad_fn=<MulBackward0>)
22nsor(0.7257, grad_fn=<MulBackward0>)
23nsor(0.7637, grad_fn=<MulBackward0>)
24nsor(0.2066, grad_fn=<MulBackward0>)
25nsor(0.5460, grad_fn=<MulBackward0>)
26nsor(0.7222, grad_fn=

KeyboardInterrupt: 

In [29]:
# Collect the latents and stdardize it.
latents = []
latent_sapce = []
for x in test_dataloader:
    _,mu,var = netG(x)
    mu = mu.detach().cpu().numpy()
    for i in range(mu.shape[0]):
        latent_sapce.append(mu[i])
        
print('latent_space finish')
latent_space = np.asarray(latent_sapce)

print(latent_space.shape)
latents = (latent_space - np.mean(latent_space, axis=0)) / np.std(latent_space, axis=0)

# Use PCA to lower dim of latents and use K-means to clustering.
pca = PCA(n_components=32, copy=False, whiten=True, svd_solver='full')
latent_vec = pca.fit_transform(latents)
latent_vec = TSNE(n_components = 3,verbose=1).fit_transform(latent_vec)
result = KMeans(n_clusters=2, random_state=2, max_iter=1000).fit(latent_vec).labels_


"""
latents = PCA(n_components=16).fit_transform(latents)
result = KMeans(n_clusters = 2).fit(latents).labels_
"""
# We know first 5 labels are zeros, it's a mechanism to check are your answers
# need to be flipped or not.
print(np.sum(result[:5]))
if np.sum(result[:5]) >= 3:
    result = 1 - result
""""
if np.sum(result[:5]) != 0 or np.sum(result[:5])!=5:
    print("redo")
"""
# Generate your submission
df = pd.DataFrame({'id': np.arange(0,len(result)), 'label': result})
df.to_csv('baseline_11_15_03_30.csv',index=False)

latent_space finish
(9000, 512)
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 9000 samples in 0.007s...
[t-SNE] Computed neighbors for 9000 samples in 4.172s...
[t-SNE] Computed conditional probabilities for sample 1000 / 9000
[t-SNE] Computed conditional probabilities for sample 2000 / 9000
[t-SNE] Computed conditional probabilities for sample 3000 / 9000
[t-SNE] Computed conditional probabilities for sample 4000 / 9000
[t-SNE] Computed conditional probabilities for sample 5000 / 9000
[t-SNE] Computed conditional probabilities for sample 6000 / 9000
[t-SNE] Computed conditional probabilities for sample 7000 / 9000
[t-SNE] Computed conditional probabilities for sample 8000 / 9000
[t-SNE] Computed conditional probabilities for sample 9000 / 9000
[t-SNE] Mean sigma: 1.197040
[t-SNE] KL divergence after 250 iterations with early exaggeration: 92.095146
[t-SNE] KL divergence after 1000 iterations: 2.123506
3
